# Data Visualization Final Project - Global Population Changes 1950 - 2100
October 2024, Portugal

*Your final project post should include:* 

 - A brief recap of your data, goals, and tasks, focusing on those that most directly influence your design
 - Screenshots of and/or a link to your visualization implementation (see below for additional guidance)
 - A summary of the key elements of your design and accompanying justification
 - A discussion of your final evaluation approach, including the procedure, people recruited, and results. Note that, due to the difficulty of recruiting experts, you can use colleagues, friends, classmates, or family to evaluate your designs if experts or others from your target population are unavailable. 
 - A synthesis of your findings, including what elements of your approach worked well and what elements you would refine in future iterations.


## Data, Goals and Tasks

### Goals

I decided to use the UN population data for my visualization project. Mainly because I was surprised to discover that far from having a population growth crisis, for many countries population is already peaking. For the world as a whole, various scenarios suggest the tipping point will likely be in the next 30 - 60 years. My over-arching goal for the visualization is to provide a tool that will let users quickly explore the population changes and understand where they are happening and which ones are significant. There are visualizations available on the web, the ones I looked at focused on precise selection of region of interest and gave clear line charts showing trends (and in some cases potential projection confidence intervals). I did not find a tool that allowed a rapid comparison between regions, so this became my focus.

Example chart from United Nations website. Strong on trend with good information on confidence intervals. But technical and, for the average user, hard to use to compare regions.

![World Population Trend from UN](World.png)

As with all the projects I have done so far on the MS-DS course, my primary objective is learning. As such, I am less concerned with the niceties of the data-set and the functionality of the visuals, than: did I learn more about the method? did I try to apply the tools and frameworks? have I improved my ability to code such visualizations? 

### Data

Link for UN Population datasets: https://population.un.org/wpp/Download/Standard/CSV/

There are many datasets to choose from. Keep it simple for this project, because: (1) 5000 record limitation in Altair package (without work-rounds) (2) the goal is to learn about visualization, not data wrangling.  I used the "Demographic Indicators 1950-2100, medium" data. The resulting datafile was still significantly larger than Altair can handle. The following filters were applied to make it manageable:
  - Single year increments changed to decade increments, given the timeframe for population growth this is also better suited to my goals
  - Limited to one growth scenario: "Medium". With more time: it would be interesting to remove this filter to show the range of projections.  

Other data preparation was limited to:
  - ensuring there are two valid levels for exploration 'Location' = Country and 'Region' - containing several countries
  - handling missing values: North America was missing as parent region for several countries.  

Basic stats about the resulting dataframe are shown below.

In [1]:
import pandas as pd
import altair as alt
dtype_dict = {'Notes': 'str', 'ISO3_code': 'str', 'ISO2_code': 'str', 'LocTypeName': 'str', 'Location': 'str', 'Variant': 'str'}
data= pd.read_csv("data/WPP2024_TotalPopulationBySex 2.csv", dtype=dtype_dict)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720210 entries, 0 to 720209
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   SortOrder    468660 non-null  float64
 1   LocID        720210 non-null  int64  
 2   Notes        113880 non-null  object 
 3   ISO3_code    346020 non-null  object 
 4   ISO2_code    344560 non-null  object 
 5   SDMX_code    416100 non-null  float64
 6   LocTypeID    468660 non-null  float64
 7   LocTypeName  468660 non-null  object 
 8   ParentID     468660 non-null  float64
 9   Location     720210 non-null  object 
 10  VarID        720210 non-null  int64  
 11  Variant      720210 non-null  object 
 12  Time         720210 non-null  int64  
 13  MidPeriod    720210 non-null  float64
 14  PopMale      720210 non-null  float64
 15  PopFemale    720210 non-null  float64
 16  PopTotal     720210 non-null  float64
 17  PopDensity   720210 non-null  float64
dtypes: float64(9), int64(3),

In [2]:
# Basic filtering to create master dataframe for exploration
location_list = data.filter(items=['LocID','Location']).groupby('LocID').first()
location_list.rename(columns = {'Location':'Parent'}, inplace = True)
df = data.join(location_list, on='ParentID')
df = df.filter(items = ['LocTypeName','Parent','Location','Variant','Time','PopMale','PopFemale','PopTotal','PopDensity'])
df.rename(columns = {'Time':'Year', 'Parent':'Region'}, inplace = True)
df['Year'] = df['Year'].astype(int)
# Handle missing values: Parent missing - > Parent = North America
df['Region'] = df['Region'].fillna('North America')
# Filter to create nice simple data set to work with
df = df[(df['LocTypeName']=='Country/Area') 
        & (df['Variant']=='Medium') 
        #& (df['Region'].str.contains('Europe'))
        & (df['Year'] % 10 == 0) # to keep data size less than 5000 Altair limitation for now
        #& (df['Year'] > 1949)
        #& (df['Year'] < 2101)
        ]
df.describe()

,Year,PopMale,PopFemale,PopTotal,PopDensity
count,3792.000000,3792.000000,3792.000000,3.792000e+03,3792.000000
mean,2025.000000,15638.281584,15580.881153,3.121916e+04,400.292772
std,46.103802,60624.669654,58721.899052,1.193336e+05,1922.591394
min,1950.000000,0.252000,0.276000,5.280000e-01,0.049000
25%,1987.500000,155.877750,157.479750,3.153508e+02,27.551000
50%,2025.000000,2069.407500,2093.354000,4.167961e+03,85.385500
75%,2062.500000,9527.679750,9615.295000,1.914680e+04,216.501500
max,2100.000000,864648.852000,836302.862000,1.700952e+06,31871.812000


### Tasks

The intended users of this visualization are casual browswers of the population information. As such it is important that the tool be easy to used and support simple exploration of the data. Using the framework suggested in the lectures:  

*Why is the task pursued?(goal)*: To explore the population data available in order to better understand trends at country/regional level.  

*How is the task conducted?(means)*: Support drill-down from global to regional level, navigate from region to region, to facilitate comparison. Organise different views to understand the twin aspects of: (1) change over time, (2) contribution to the whole (how does each country impact the global number.  

*What does a task seek to learn about the data?(characteristics*: This is definitely about trends and high-level comparisons. There is no requirement to compare specific values or pull out individual data points.  

*Where does the task operate?(target data)*: The dataset has an absolute time-frame 1950-2100. In future evolutions, the tool should support relative comparisons between growth scenarios.  
 
*When is the task performed?(workflow)*: As envisaged, there is not a significant prior workflow (casual browsers). In design, I imagined that users would typically enter at the top level and follow various paths of exploration according to interest.  
 
*Who is executing the task?(roles)*: One user type: "casual browser"  


### Visualizations Developed and Justification for Key Design Elements

Key Design Element decisions are <u>underlined</u> throughout this section.  

My first attempt was to create a <u>stacked bar chart</u>, in order to give an overview of the global population change between 1950 and 2100. There were other options for the chart format such as line or area. However, the stacked bar provides a couple of visual cues to the data that I feel are helpful: (1) separation of each decade's data, whilst retaining the shape of the evolution, (2) best clarity in terms of colour of regions to help perceive the detail. 

I experienced significant challenges with <u>ordering the stacking</u> of the regions in a way that kept geographically close regions together, without which the data was very hard to interpret. Finally solved through using "transform_calculate" an Altair functionality that was new to me.   <u>Colour</u> is used to help the user separate out the regions quickly. I was not able to get to a colour scheme that really suited my purpose. "Spectral" was the closest I found, because it helped separate individual regions, but also helped group regions into meaningful geographical areas. However, the differences with how we perceive darker colours (the reds and greens) vs the lighter colours (yellows and oranges) mean the overall effect is not quite what the design intended. Definitely on the list for future improvement.

<u>Selection interactivity</u> was added to help the user focus in on a specific region of interests, by dimming the unselected regions. In initial tests, it became clear that this didn't really provide sufficient information to understand the regional perspective. Especially, for smaller regions that become hidden given the massive changes in Africa, India and China. 

<u>Tooltips</u> provided some additional info to the user and are helpful in identifying specific countries on the chart. In this initial prototype, this is the only method to get at individual country data. This was a trade-off between complexity (too many countries to see clearly) and losing the information completely (e.g. through a groupby region data source). In a future version, a third level of drill-down would be desirable.

This interactivity helps exploration but does not solve the challenge that trend information is hard to perceive. 

#### Prototype 1 - Global Trend

**Hover the mouse over the chart to see Country Names and Populations within a region**  
**Click on the legend to select a particular region of interest**

In [10]:
sorted_regions = "{'Eastern Africa':0, 'Middle Africa':1, 'Northern Africa':2, 'Southern Africa':3, 'Western Africa':4, 'Central Asia':5, 'Eastern Asia':6, 'Southern Asia':7, 'South-Eastern Asia':8, 'Western Asia':9, 'Eastern Europe':10, 'Northern Europe':11,'Southern Europe':12, 'Western Europe':13, 'Caribbean':14,'Central America':15, 'South America':16, 'North America':17,'Australia/New Zealand':18, 'Melanesia':19, 'Micronesia':20, 'Polynesia':21}[datum.variable]"

selection = alt.selection(type='multi', fields=['Region'], bind='legend')

chart_one = alt.Chart(df).transform_calculate(
    order = sorted_regions
    ).mark_bar(size=20).encode(
    x = alt.X("Year", scale=alt.Scale(domain=[1950, 2100]), axis=alt.Axis(title='Year', format=".0f")),
    y = alt.Y("sum(PopTotal)", title='Total Population 000s'),
    color=alt.Color('Region',sort=alt.SortField("order", "descending"), scale=alt.Scale(scheme='spectral')),
    order = "order:O",
    tooltip=alt.Tooltip(["Location:N"]),
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).properties(
    width=500,
    height=400,
    title='Global Population 1950 - 2100',
).add_selection(selection)

chart_one

alt.Chart(...)

In the second iteration of this visualization (Combination Chart One - see section below), I explored the <u>extract/elaborate</u> functionality that we looked at in the lecture example. For the detail chart, I wanted to help the user understand the trend for a specific region whilst retaining the context of how this contributes to the global picture. In order to keep this clear, I used a <u>line chart</u> for the region (no clutter to distract).

I do think this gives the user a much clearer picture of how each region is changing over time. The changing y-axis concerned me a little, as the scale when looking at Southern Asia is vastly different compared to Polynesia. As the task objective was to understand "trend" and not to make low level quantitative comparisons, I believe the result is not misleading and does greatly clarify the perception of change over time.


Combination Chart One allows reasonable exploration of trend, however it still was not easy to compare regions one to another. I developed a second prototype that shows 'region' on the x-axis and allows the user to move through time using an <u>interactive slider</u>. Whereas, the line chart provides great clarity on trend, this <u>animated bar-chart</u> does a reasonable job of comparing the relative contribution of each region at any point in time. Further, I think the animation actually helps to build the understanding of how this relative importance is changing over time. As such, I think it is an example of where animation improves rather than hinders usability. 

The <u>colour scheme is maintained</u> with consistent meaning throughout the visualizations, which is intended to help the user retain information from one to the other.  

There was a remaining niggle, despite the tooltips, it is fairly cumbersome (if not impossible) to get at the country level of detail. And, I suspect many users would want to know the composition of a region to better understand what was going on. This led me to my second version of the contribution histogram (Combination Chart Two), that allows the user to select a region (this time by clicking on the column rather than selecting the legend) and this will show the equivalent histogram for the countries within that region on the right hand chart.

I chose to allow clicking on the columns rather than legend, principally because I wanted to find out which users found easier to use (user evaluation subsequently showed that the placement of the legend on the right was a challenge for some who failed to see that it linked to both charts).  This allowed me to delete the legend, which is duplicate information as the regional information is clear in the histogram. I would also like to have carried over the colour from the regional overview to the country histogram, but I've not yet worked out how to do that. Lastly, this iteration allowed me to change the tooltip for the regional chart, instead of showing the detail element (country) it shows the regional information, potentially less confusing for users.

One unsolved issue, the scale on the rhs changes as the year changes, which I feel can be misleading. Needs addressing in a future release, although it doesn't seem simple to do so in Altair.

#### Prototype 2 - Relative Contribution

**Use the slider below the chart to change the Year**

In [5]:
# Using a slider to visualize how regional population share of global is changing over time
slider = alt.binding_range(min=1950, max=2100, step=10)
year = alt.selection_single(name="year", fields=['Year'], bind=slider, init={'Year':2020})

hist_chart = alt.Chart(df).transform_calculate(
    order = sorted_regions
    ).mark_bar(size=20).encode(
    x = alt.X("Region", sort=alt.SortField("sum(PopTotal)", "descending")),
    y = alt.Y("sum(PopTotal)", title='Total Population 000s', scale=alt.Scale(domain=[0, 2500000])),
    color=alt.Color('Region',sort=alt.SortField("order", "descending"), scale=alt.Scale(scheme='spectral')),
    order = "order:O",
    tooltip=alt.Tooltip(["Location","PopTotal"]),
).properties(
    width=500,
    height=400,
    title='Regional Contribution to Global Population 1950 - 2100',
).add_selection(
    year
).transform_filter(
    year
)
hist_chart

alt.Chart(...)

<br />
<center>-----------------------------------------------------------------------------------------------------</center>


#### Combination Chart One - Trends in Population Growth Globally and by Region

**Hover over the chart to see Country Name and Population**  
**Click on the legend to select a particular region of interest - this will change the right hand chart to focus on that region**

In [5]:
# First attempt at extract/elaborate

# filtering using dynamic queries. 
selection = alt.selection(type='multi', fields=['Region'], bind='legend')

# Create a container for our two different views
base =  alt.Chart(df).properties(width=900, height=400)

# Let's specify our overview chart
overview = alt.Chart(df).transform_calculate(
    order = sorted_regions
    ).mark_bar(size=20).encode(
    x = alt.X("Year", scale=alt.Scale(domain=[1950, 2100]), axis=alt.Axis(title='Year', format=".0f")),
    y = alt.Y("sum(PopTotal)", title='Total Population 000s'),
    color=alt.Color('Region',sort=alt.SortField("order", "descending"), scale=alt.Scale(scheme='spectral')),
    order = "order:O",
    tooltip=alt.Tooltip(["Location","PopTotal"]),
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).add_selection(selection).properties(
    height=400, 
    width=450,
    title='Global Population 1950 - 2100',
)

# Create a detail chart
detail = base.mark_line().encode(
    y = alt.Y("sum(PopTotal)", title= None), 
    x = alt.X("Year", scale=alt.Scale(domain=[1950, 2100]), axis=alt.Axis(title='Year', format=".0f"))
).transform_filter(selection).properties(
    height=400, 
    width=450,
    title='Trend for Selected Region',
)

overview | detail

alt.HConcatChart(...)



<br />
<center>-----------------------------------------------------------------------------------------------------</center>

#### Combination Chart 2 - Chaging Contribution to Global Population by Region and Country

**Click on a column to select a specific region - this will show the countries that make up that region on the right hand chart**  
**Use the slider below the chart to change the Year**

In [26]:
# filtering using dynamic queries. 
selection = alt.selection(type='multi', fields=['Region'])
slider = alt.binding_range(min=1950, max=2100, step=10)
year = alt.selection_single(name="year", fields=['Year'], bind=slider, init={'Year':2020})

# Create a container for our two different views
base =  alt.Chart(df).transform_calculate(
    order = sorted_regions
    ).properties(
    width=900, 
    height=400
).add_selection(
    year
).transform_filter(
    year
)

# Let's specify our overview chart
hist_chart = base.mark_bar(size=20).encode(
    x = alt.X("Region", sort=alt.SortField("sum(PopTotal)", "descending")),
    y = alt.Y("sum(PopTotal)", title='Total Population 000s', scale=alt.Scale(domain=[0, 2500000])),
    color=alt.Color('Region',sort=alt.SortField("order", "descending"), scale=alt.Scale(scheme='spectral'), legend=None),
    order = "order:O",
    tooltip=alt.Tooltip(["Region","sum(PopTotal)"]),
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).properties(
    width=500,
    height=400,
    title='Regional Contribution to Global Population 1950 - 2100',
).add_selection(selection)
    
    
# Create a detail chart
country_detail = base.mark_bar().encode(
    y = alt.Y("sum(PopTotal)", title= None), 
    x = alt.X("Location"),
    tooltip=alt.Tooltip(["Location","PopTotal"])
).transform_filter(selection).properties(
    height=400, 
    width=400,
    title='Countries in Selected Region',
)

hist_chart | country_detail

alt.HConcatChart(...)

<br />
<center>-----------------------------------------------------------------------------------------------------</center>
<br />
## Evaluation

The goal of the visualization is to provide an exlporatory tool that allows a user to better understand the trends and relative contributions of specific regions to global population growth. Therefore, I was keen to understand if users (other than myself) were able to use the tool to find insights they found interesting.

Although this is being done as part of the final project, I will still describe the planned evaluation as formative rather than summative. At best, this V1 of my visualization tool can be considered a hifi prototype. It certainly isn't yet a finished product. The evaluation should provide useful information into aspects of the design that need to be reworked if and when I generate a V2. 

### Evaluation Plan

The task briefing in the lecture series encouraged us to break down the evaluation using the following key questions:

*The target question you want to answer*: Are you able to improve your understanding of global population growth through this tool and do you find new insights that provoke interest and follow-up? How could the tool better meet your needs for exploration of the data set?  

*The people you would recruit to answer that question*: Casual browsers of the data, my intent is to provide a tool for exploration to those with an interest, as opposed to a tool for experts to really dive into the specifics. Further, given this is an academic exercise rather than a real world study, it was more important to scope the task around available participants.  I've kept them anonymous in the report, but drew from close friends and family, none of whom are data science professionals. 

*The kinds of measures you would use to answer your data (e.g., insight depth, use cases, accuracy) and what these measures would tell you about the core question*: Primarily I am looking to capture insights. So I am interested in: the questions users ask, any spontaneous insights they have, what they find surprising or interesting. However, I would also like to capture what they wanted to know, but couldn't find out (and whether usability related or functionality/data related). Thirdly, I would like findings that either confirm usability or help to guide changes in usability design in future iterations. Key metrics: time to first insight, number of insights, qualitative feedback and post usage commentary.  

*The approach you will use to answer that question (e.g., a journaling study, a formal experiment, etc.)*: Given the simplicity of the tool and the nature of the questions/metrics desired, it is most appropriate to use a combination of Think-Aloud study and semi-structured interview.  

*How you would instantiate those methods (i.e., what would your participants do?)*: Here is the planned evaluation session:
1) Standard introduction to the purpose of the evaluation and the data source.
2) 2 - 3 minutes to play with the tool without introduction: is it intuitive? does the user find the mechanics of the interaction without instruction?
3) 3 - 5 minutes of think-aloud, guided by three questions posed up front: (1) what can you tell me about changes in global population? (2) what do you learn that is either surprising or interesting to you? (3) what did you want to know more about, but were unable to find out?
4) Closing questions: (1) did you enjoy using the tool? why? or why not? (2) do you have any feedback about the usability? (3) what would you like to see in a future version?  

*What criteria would you use to indicate that your visualization was successful*: (1) Users find at least one insight they state as interesting or surprising, (2) the overall feedback regarding using the tool is net positive.

### Evaluation Results Summary

For detailed evaluation results, see Appendix 1.

I asked the users to evaluate "Chart Combination One" and "Chart Combination Two". As it was my intent that these two chart combos be used together to understand trend and contribution.

Here is a bulleted list of my key takeaways from the user evaluation sessions. Some of these are obvious, but useful reminders. Others I already knew but couldn't fix. A few were genuinely new to me. The most important key learning: when you are developing something, you become very familiar with it, which in turn means that you simply cannot see the usability challenges that new users face. **User Evaluation is essential and provides unexpected new learning.** 

*Positives*
* Overall positive reactions to the tool and the information insights they were able to glean
* I learned something and enjoyed the session
* More interest in the second chart combo - Users were more interested in individual countries and liked the animation of the histogram.
* Multiple comments that the line trend graph was the easiest to interpret
* Users were surprised at how starkly different the population trends are in different parts of the world - a key insight
* Overall positive to look and feel and usability

*Challenges to resolve*
* Not as intuitive as I had hoped. These charts would need clear presentation and introduction to make them usable without guidance.
* Users don't look very far away from their focus of interest (for example: didn't see the instructions and couldn't relate the legend to the left hand chart)
* Usability tweaks were distracting (for example: number formatting in tooltips), frustrating because these were things I could not find a way to correct in Altair.
* Users didn't move between the chart combos as I had intended. Tending to remain stuck on a particular way to explore the data.
* Regions did not have enough semantic meaning to be a useful summary level for multiple users. 
* Users wanted to get to numbers, even though that was not the purpose of the tool. (but this is how many of us think)
* Colour is dangerous, it has unintended meaning (for example: red = warning), that was an issue for one user.



## Findings and Conclusions

The combination of the three charts achieves the objectives of providing a good overview of how population is changing whilst allowing the user to dive into regional trends and to better understand relative contribution. 

In the lectures we were encouraged to judge visualization by the insights generated. For me there were several new insights, most notably:
   - How population growth is leveling off, creating a different context for economic growth in the coming decades.    
   - How different the population trends are in differing parts of the world, prompting me to want to dive into the reasons underlying these changes
   - How stark the declines are in Eastern Asia (China) and that Europe is already stagnant, this will likely cause a significant shift in global wealth in the coming decades.
   - The massive growth in Africa, which will exceed China by 2100 only India will be larger. How will that impact the world?

With regard to my learning objectives, I am pleased with what I learnt about Altair. Though it became clear to me there is a lot more to learn and I need to find additional tutorial content. The method, tools and frameworks still feel a bit 'heavy' to me. I was able to use them, but I didn't think it was giving me much assistance with my thought process. This is potentially due to the light prototype nature of the simple tool I built. Such methodology is over-kill. However, the project reinforced the learning, refreshed the definitions and is a useful reference when I need to build something in the wild. The aspect I need to focus on is the transition from lo-fi to hi-fi prototype. I am uncomfortable spending much time in the lo-fi step (skills with paper and pencil lacking), then find the development of something functional limited more by what I can achieve in code than it is by the design concept I had in mind. I buy the usefulness of doing both steps, it is an area for improvement going forward.


###  Where next with the visualization?

#### Grouping and drill-down

The regional grouping provided in the data set is not ideal. There are 18 regions which is perhaps too many for a normal user to keep in their head. The regional make-up is not always clear (is India in South or Sout-East Asia??). In a future version, I would like to solve this confusion using three levels of drill-down:  
1) Global  
2) Continent (North America, South America, Europe, Asia, Australasia, Africa)  
3) Major Country  - meaning individual country where population in 2020 is > 5m and grouped countries(regions) below this population. 

Spending time on getting to a clear set of levels that are intuitive to the user whilst allowing consistent drill-down will help usability. Simplification, (for example, by removing smaller countries) would also help.

#### Design Changes

There are a couple of design changes I would like to make:  
1) Optimise the colour scheme to maintain the clarity of regional separation, but avoid the problem that light colours are so much harder to differentiate.  
2) Small tweaks that are more about familiarity with Altair: e.g. how to round the population information in the tooltips.  
3) Get the titles to update based on selection, to give the user more help with context of the task.  (seemed hard in Altair documentation)
4) The changing axis scale on the right hand chart in combo 2 is a problem. I could not see a way to fix it with my current knowledge of Altair.

#### Scenarios

The UN provides several population growth scenarios with differing underlying assumptions. Given the time, I would like to have created a facet wrap visualization to be able to compare and contrast these scenarios.

#### Confidence Intervals

Much of this visualization is about the future. Clearly there are potential errors in such population forecasts. It would have been interesting to understand the confidence intervals around the trend information. If I had been able to find the data this would have been easy to add to my trend chart without cluttering the visual. 

I believe this information is vital to avoid jumping to conclusion that may not be supported by the data.

 <br />
 <br />

## Appendix 1 - Detailed Evaluation Results

#### Participant A

*Play with the visualisation: is it intuitive?*

Quite intuitive, but not immediately so. Needed some initial guidance.
   - Didn't read the instructions at top of chart
   - Confused by separation of legend from first chart in combo
   - Initially confused by colour scheme and attributed meaning to red/green that was unintended.
   - Concerned about changing axis scale on right hand side
   - Wanted to see more than one trend on right hand side
   - Confused by region definitions and felt there were too many.
   - on second chart combo, slider was hidden on user's screen


*Think-aloud usage (i.e. you tell me what you’e doing), guided by three questions:* 
			(1) what can you tell me about changes in global population?
			 - Not all regions are following the same path (had assumed it would do)
			(2) what do you learn that is either surprising or interesting to you?  
			 - Stark decline in China
			 - India and Africa overtakes China
			 - States continues to grow (due to immigration?)
			(3) what did you want to know more about, but were unable to find out?
			 - not enough time to reach this point in my exploration

*Closing questions:* 
			(1) did you enjoy using the tool? why? or why not?  
			 - Really liked the second chart combo - able to see individual countries  
			 - Found the trend information interesting  
			(2) do you have any other feedback about the usability?
			 - Would prefer instructions located on the chart itself (missed them at first glance)
			 - Difficult to compare regions on rhs, because the axis-scale was changing  
			(3) what would you like to see in a future version?
			 - simplify the regional structure (why is Polynesia distinct, but ANZ is together)  


#### Participant B

*Play with the visualisation: is it intuitive?*

Quite intuitive, but not immediately so. Needed some initial guidance.
   - global population chart hard to read, wanted to have detail information (too abstract)  
   - shows growth of each country  
   - population numbers in tool tip should be in millions (simplify units)  
   - when you click on a region it becomes interesting  
   - blue curve is more interesting than lhs  
   - on second chart combo, didn't like that couldn't read all the countries in global view  
   - tooltips for countries in first chart combo were confusing - the numbers need to be in millions  


*Think-aloud usage (i.e. you tell me what you’e doing), guided by three questions:* 
			(1) what can you tell me about changes in global population?  
			  - reacted surprised on seeing the overall trends in various regions  
			(2) what do you learn that is either surprising or interesting to you?  
			 - surprised that some countries are seeing population decline  
			 - knew from economics that we didn't have a sufficient replacement factor, but was surprised how soon it was happening  
			 - Africa vs China will be a dramatic change  
			(3) what did you want to know more about, but were unable to find out?
			 - nothing really  

*Closing questions:* 
			(1) did you enjoy using the tool? why? or why not?
			 - Really liked the second chart combo - able to see individual countries  
			 - Found the trend information interesting  
			 - if I had a reason to use the information, the tool was fun to use.  
			(2) do you have any other feedback about the usability?  
			 - return from selecting one region to selecting all is not obvious.  
			 - number formatting in tooltips, slider format, could be improved  
			(3) what would you like to see in a future version?  
			 - animation of the changes through the years (play a video for the user)  
			 - Would want to be able to get the actual data  

#### Participant C

*Play with the visualisation: is it intuitive?*

Quite intuitive, but not immediately so. Needed some initial guidance.
   - Didn't read the instructions at top of chart
   - Confused by separation of legend from first chart
   - Initially confused by colour scheme and attributed meaning to red/green that was unintended.
   - Concerned about changing axis scale on right hand side
   - Wanted to see more than one trend on right hand side
   - Confused by region definitions and felt there were too many.


*Think-aloud usage (i.e. you tell me what you’e doing), guided by three questions:* 
			(1) what can you tell me about changes in global population?  
			(2) what do you learn that is either surprising or interesting to you?   
			(3) what did you want to know more about, but were unable to find out?  

*Closing questions:* 
			(1) did you enjoy using the tool? why? or why not?
			 - Really liked the second chart combo - able to see individual countries
			 - Found the trend information interesting  
			(2) do you have any other feedback about the usability?  
			(3) what would you like to see in a future version?  

#### Participant D

*Play with the visualisation: is it intuitive?*

Quite intuitive, but not immediately so. Needed some initial guidance.
   - Didn't read the instructions at top of chart
   - Confused by separation of legend from first chart
   - Initially confused by colour scheme and attributed meaning to red/green that was unintended.
   - Concerned about changing axis scale on right hand side
   - Wanted to see more than one trend on right hand side
   - Confused by region definitions and felt there were too many.


*Think-aloud usage (i.e. you tell me what you’e doing), guided by three questions:* 
			(1) what can you tell me about changes in global population?  
			(2) what do you learn that is either surprising or interesting to you?  
			(3) what did you want to know more about, but were unable to find out?

*Closing questions:* 
			(1) did you enjoy using the tool? why? or why not?  
			 - Really liked the second chart combo - able to see individual countries  
			 - Found the trend information interesting  
			(2) do you have any other feedback about the usability?  
			(3) what would you like to see in a future version?  
